<a href="https://colab.research.google.com/github/monku12/Dialogue-act-classification/blob/main/CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Embedding, Bidirectional, LSTM
import numpy as np

In [ ]:
!pip install tf-crf-layer

In [8]:
from tf_crf_layer.layer import CRF
from tf_crf_layer.loss import crf_loss
from tf_crf_layer.metrics import crf_accuracy

In [9]:
vocab = 3000
EMBED_DIM = 300
BiRNN_UNITS = 48
class_labels_number = 9
EPOCHS = 10

In [10]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [12]:
import pandas as pd
data = pd.read_csv('train.csv')
print(data.shape)
data.head(3)

(75066, 6)


,Unnamed: 0,Speaker,Utterances,Basic,General,Full
0,0,fe016,so um,F,fh,fh
1,1,fe016,i was going to try to get out of here like in ...,S,s,rt
2,2,fe016,um,F,fh,fh


In [13]:
data=data.drop(columns=["Unnamed: 0"])
data.head(3)

,Speaker,Utterances,Basic,General,Full
0,fe016,so um,F,fh,fh
1,fe016,i was going to try to get out of here like in ...,S,s,rt
2,fe016,um,F,fh,fh


In [15]:
import re

data['Utterances'] = data['Utterances'].apply(lambda x: x.lower())
data['Utterances'] = data['Utterances'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
data['Utterances'].head(3)

0                                                so um
1    i was going to try to get out of here like in ...
2                                                   um
Name: Utterances, dtype: object

In [18]:
#Creating table of number of each data label present in the dataset
label_count=data.groupby('Basic')['Basic'].agg('count')
label_count.to_numpy(dtype=int)
label_count = np.expand_dims(label_count, axis=0)
label_count.shape=(5,1)
labels=np.array(['B', 'D', 'F', 'Q', 'S'])
labels.shape=(5,1)
label_count=np.column_stack((labels,label_count))

df = pd.DataFrame(label_count, columns=['Label', 'Count'])

convert_dict = {'Label': str,
                'Count': int
               }
  
df = df.astype(convert_dict)
df

,Label,Count
0,B,10265
1,D,9739
2,F,5323
3,Q,4640
4,S,45099


In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Tokenizing the words and padding the sentence
MAX_PADDING = 100
tokenizer = Tokenizer(num_words = 5000, split=" ")
tokenizer.fit_on_texts(data['Utterances'].values)
x_train = tokenizer.texts_to_sequences(data['Utterances'].values)
x_train = pad_sequences(x_train, padding='post', maxlen=MAX_PADDING)
print(x_train.shape)
x_train[:2]

(75066, 100)


array([[ 11,  16,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,  36,  58,   4, 170,   4,  61,  83,  10,  96,  34,  15, 441,
         87, 741,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [22]:
y_train = pd.get_dummies(data['Basic']).values
[print(data['Basic'][i],y_train[i]) for i in range (0,5)]

F [0 0 1 0 0]
S [0 0 0 0 1]
F [0 0 1 0 0]
S [0 0 0 0 1]
S [0 0 0 0 1]


[None, None, None, None, None]

In [26]:
model = Sequential()
model.add(Embedding(5000, 256, mask_zero=True)) 
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(CRF(5))

model.compile('adam', loss=crf_loss, metrics=[crf_accuracy])
model.fit(x_train, y_train, epochs=1)

<tf_crf_layer.layer.CRF object at 0x7f6ecdf3e550> 0


ValueError: ignored